#### Numerical solutions of ordinary differential equations

Suppose we have a gen $A$, which suppresses its own expression, when its expression level is too high. To keep things simple, let's also assume linear dependence between the expression _rate_ $\frac{dc_{A}}{dt}$ and expression _level_ $c_{A}$: 

<img width="300" height="50" src="../images/ode_negative_feedback.png"></img>

This system is boring, but it has one distinctive advantage - it has simple analytical solution:

$$ \frac{dc_A}{dt} = -kc_{A} \rightarrow 
   \int_{c_{A0}}^{c_{A}}\frac{dc_{A}}{c_{A}} = -k\int_{0}^{t} dt \rightarrow
    ln(c_{A}) - ln(c_{A0}) = -kt \rightarrow
    c_{A} = c_{A0}e^{-kt}$$
where $c_{A}$ is expression level of $A$ at time $t$, and $c_{A0}$ is expression level of $A$ at time $t=0$

Now, when we have the analytical solution, we can use it as a ground truth to assess the 'wellness' of numerical solution. For now we'll be using [Euler Forward](https://en.wikipedia.org/wiki/Euler_method) method for numerical integration. Surely, it's not the most efficient one, but it has a very simple intuition behind it, which is definitely a plus: 

>We first discretize time into small intervals $\Delta t$. If these intervals are small enough, we can assume, that the change in our function $\Delta c_{A}$ over each of these intervals $\Delta t$ is linear, or, in other words, within each of these intervals function $c_{A}$ has a constant slope:  $\frac{\Delta c_{A}}{\Delta t} = \text{constant}$. What is the value of this slope? Since we assumed that the slope over $\Delta t$ is constant, it would be sufficient to 'sample' the slope at one point on the interval. In Euler Forward we sample the slope value at the _beginning_ of the interval:

> $$\frac{\Delta c_{A}}{\Delta t} = \frac{dc_{A}}{dt} \rightarrow 
    \frac{c_{A}(t+\Delta t) - c_{A}(t)}{\Delta t} = \frac{dc_{A}(t)}{dt} \rightarrow
    c_{A}(t+\Delta t) = c_{A}(t) + \frac{dc_{A}(t)}{dt} \Delta t $$
> where $\frac{dc_{A}}{dt}$ comes from the mass balance (or any other balance). 

And there we have it - iterative relation for $c_{A}$, which only requires the expression for $ \frac{dc_A}{dt}$ (that's $ \frac{dc_A}{dt} = -kc_{A}$) and the value of $c_{A}$ at time $t=0$ (initial condition). 

>__Note__: in the code below we'll try to comply with notation conventions of [state-space](https://en.wikipedia.org/wiki/State-space_representation) representation; we'll use:
- $x$ to denote the properties of the system, that characterize its _state_ at any time instant ($c_{A}$), 
- $u$ to denote all external inputs to the system (in our simple example there are no inputs),
- $p$ for all the system parameters (in this example there's only one $p = -k$).

In [1]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.widgets import Div
from bokeh.layouts import row, column
from  ipywidgets import interact
#import warnings
#warnings.filterwarnings('ignore') 
output_notebook()
#!jupyter nbextension enable --py widgetsnbextension

# ---define the system---
# mass balance: all we know about the system is stored here
def balance(x, p):
    """dcdt = -kc"""
    k = -p[0] # p = -k
    return -k*x 

# initial condition
x0 = 1

# time related
t0,dt,tf = 0,0.1,10
t_span = np.arange(t0,tf+dt,dt)

# parameters
p = [-1]    

# ---solve analytically---
def analytical(x0, t_span, p):
    return [x0*np.exp(p[0]*t) for t in t_span]

# ---solve numerically (Euler Forward)---
def eulerForward(dxdt, x0, t_span, p):  
    """x[i+1] = x[i] + dxdt[i]*dt"""
    dt = t_span[1] - t_span[0]

    x_span, x = [x0], x0
    for t in t_span:
        x += dxdt(x, p)*dt
        x_span.append(x)

    return x_span[:-1] # on each iter we calculate x for the next step, so at the end we'll have one extra x  

# ---plot stuff!---
plt = figure(title="simple integration", x_range=[t0, tf], y_range=[0, 2*x0],
             plot_width=400, plot_height=250)
plt.xaxis.axis_label = "time"
plt.yaxis.axis_label = "expression level"
r1 = plt.line(t_span, analytical(x0, t_span, p), 
              color="grey", line_width=2, legend="analytical", line_dash=[5,2])
r2 = plt.line(t_span, eulerForward(balance, x0, t_span, p), 
              color="navy", line_width=2, legend="euler forward")

div = Div(width=400,
          text="<br>Not bad, Euler Forward,\
                <br>but how will you handle larger $\Delta t$:")
show(row(plt, div), notebook_handle=True)

# add interactive sliders to check the effect of parameters and dt
def update(p=-1, dt=0.1):
    r1.data_source.data['x'] = np.arange(t0,tf+dt,dt)
    r1.data_source.data['y'] = analytical(x0, np.arange(t0,tf+dt,dt), [p])
    r2.data_source.data['x'] = np.arange(t0,tf+dt,dt)
    r2.data_source.data['y'] = eulerForward(balance, x0, np.arange(t0,tf+dt,dt), [p])
    push_notebook()

interact(update, p=(-2,2,0.1), dt=(0.1,2,0.1))

Loading BokehJS ...

interactive(children=(FloatSlider(value=-1.0, description='p', max=2.0, min=-2.0), FloatSlider(value=0.1, description='dt', max=2.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

As expected, the system stabilizes, as long as there is a negative feedback loop (expression of $A$ is inhibited at high levels $\equiv$ $p$ value is negative). Euler Forward captures this behaviour reasonably well at small $\Delta t$, however it starts to fail miserably as $\Delta t$ increases, which is especially pronounced for systems with fast kinetics. We can get an idea of why Euler Forward is not the most fantastic integration method out there, if we check the [Taylor series](https://en.wikipedia.org/wiki/Taylor_series) expansion of the function that we're interested in ($c_{A}$) around some random point $t_{0}$:

<center><font color="black">$c_{A}(t_{0}+\Delta t) = c_{A}(t_{0}) + \Delta t \frac{dc_{A}(t_{0})}{dt}$</font> <font color="red">$ + \frac{1}{2} \Delta t^{2} \frac{d^{2}c_{A}(t_{0})}{dt^{2}} + O(\Delta t^{3})$ </font></center>

The part in <font color="black"> __black__ </font> is covered by Euler Forward (in fact, it _is_ Euler Forward), everything <font color="red">else</font> is the difference between the true solution at time instance $t_0$ and our approximation at the same time instance. We'll call this difference _local truncation error_. You can see, that local truncation error for Euler Forward scales with $\Delta t^{2}$, which is not great... 

Can we do better? Of course! And the first idea that comes to mind is ...

<font color="red">>>>mid point method, ... Runge-Kutta stuff</font>

 



In [2]:
def rungeKutta2(dxdt, x0, t_span, p):
    dt = t_span[1] - t_span[0]

    x_span, x = [x0], x0
    for t in t_span:
        k1 = dxdt(x,        p)*dt
        k2 = dxdt(x+0.5*k1, p)*dt
        x += k2
        x_span.append(x)
        
    return x_span[:-1]

# plot stuff!    
plt = figure(title="simple integration", x_range=[t0, tf], y_range=[0, 2*x0],
             plot_width=400, plot_height=250)
plt.xaxis.axis_label = "time"
plt.yaxis.axis_label = "expression level"
r1 = plt.line(t_span, analytical(x0, t_span, p), 
              color="grey",  line_width=2, legend="analytical", line_dash=[5,2])
r2 = plt.line(t_span, eulerForward(balance, x0, t_span, p), 
              color="navy",  line_width=2, legend="euler forward")
r3 = plt.line(t_span, rungeKutta2(balance, x0, t_span, p), 
              color="green", line_width=2, legend="runge kutta 2")

show(plt, notebook_handle=True)

# add interactive sliders to check the effect of parameters and dt
def update(p=-1, dt=0.1):
    r1.data_source.data['x'] = np.arange(t0,tf+dt,dt)
    r1.data_source.data['y'] = analytical(x0, np.arange(t0,tf+dt,dt), [p])
    r2.data_source.data['x'] = np.arange(t0,tf+dt,dt)
    r2.data_source.data['y'] = eulerForward(balance, x0, np.arange(t0,tf+dt,dt), [p])
    r3.data_source.data['x'] = np.arange(t0,tf+dt,dt)
    r3.data_source.data['y'] = rungeKutta2(balance, x0, np.arange(t0,tf+dt,dt), [p])
    push_notebook()

interact(update, p=(-2,2,0.1), dt=(0.1,2,0.1))


interactive(children=(FloatSlider(value=-1.0, description='p', max=2.0, min=-2.0), FloatSlider(value=0.1, description='dt', max=2.0, min=0.1), Output()), _dom_classes=('widget-interact',))

<function __main__.update>